# Tecnologías Semánticas para la Ciencia del Dato, Proyecto Final

Morales Polanco, Diego Enrique

Fecha de entrega: 24 de Enero 2024

Este proyecto consiste en la implementación de modelos de machine learning junto con métricas relacionales para clasificar las publicaciones del dataset WebKB. Este dataset consiste en 
877 publicaciones científicas clasificadas en 5 clases (curso, facultad, estudiante, proyecto, personal). Estas publicaciones están relacionadas entre si a traves de citas. Cada publicación es descrita por un vector binario de palabara, el cual indica la presencia o ausencia de una palabra en la publicación. El diccionario de palabras consiste en 1703 palabras únicas de temas relacionados a Machine Learning. Las publicaciones provienen de 4 universidades (cornell, texas, washington, wisconsin).

También se pretende comparar los resultados de esta implementación con los resultados de la investigación "Link Based Classification" por Qing Lu y Lise Getoor de la Universidad de Maryland, en la cual se utilizó el mismo dataset WebKB, con un modelo de regresión logística. 

Primero se importan las funciones y librerias a utilizar para la implementación.

In [1]:
from utils import create_graph
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RandomizedSearchCV
from sklearn.cluster import KMeans
import pprint
import warnings

Luego se prepara la data. Se creó la función "create_graphs" como función de utilidad para parsear la data de los archivos de cada universidad y crear los grafos correspondientes. Estos grafos se guardan en la variable "graphs", cada elemento de esta variable corresponde a un grafo de una universidad, el orden siendo ["cornell", "texas", "washington", "wisconsin"]. La variable word_vectors corresponde a los vectores de palabra de cada publicación de cada universidad, en el mismo orden que la variable "graphs". La variable classifications indica la clase de cada publicación de cada universidad. 

In [2]:
universities = ["cornell", "texas", "washington", "wisconsin"]
classes = {'course': 0, 'faculty': 1, 'student': 2, 'project': 3, 'staff': 4}
graphs = []
counts = []
classifications = []
word_vectors = []

for university in universities:

    cites_path = university + ".cites"
    content_path = university + ".content"

    with open(cites_path, "r") as cites_file:
        cites = cites_file.readlines()

    with open(content_path, "r") as content_file:
        content = content_file.readlines()

    g = create_graph(content, cites)
    graphs.append(g)

    count = {'course': 0, 'faculty': 0, 'student': 0, 'project': 0, 'staff': 0}
    
    y = list(nx.get_node_attributes(g,"classification").values())

    for element in y:
        count[element] += 1
    
    y = [classes[value] for value in y]

    x = list(nx.get_node_attributes(g,"content").values())
    classifications.append(y)
    word_vectors.append(x)
    counts.append(count)

Se creó la variable counts, que indica cuantas publicaciones de cada clase hay en cada universidad. Esto es para saber que tan balanceado o desbalanceado es el dataset y tomar decisiones a partir de esto.

In [3]:
counts

[{'course': 42, 'faculty': 32, 'student': 83, 'project': 19, 'staff': 19},
 {'course': 34, 'faculty': 31, 'student': 103, 'project': 18, 'staff': 1},
 {'course': 66, 'faculty': 27, 'student': 107, 'project': 21, 'staff': 9},
 {'course': 76, 'faculty': 35, 'student': 122, 'project': 22, 'staff': 10}]

Como se puede apreciar, el dataset es relativamente desbalanceado. En la universidad "texas" por ejemplo, solo hay una publicación de la clase staff, la cual también representa un porcentaje bastante pequeño de las publicaciones en las demas universidades. Se puede considerar eliminara el objeto de la universidad de "texas" perteneciente a la clase staff, ya que representa una porción no significativa de la data, pero este elemento no solo tiene un vector de palabras asociado, sino también varias conexiones a otras publicaciones. Por esto, en lugar de eliminar el, se utilizará el método de cross validation kfolds, el cuál mejora el rendimiento en problemas con datasets desbalanceados.

A continuación se calculan las métricas relacionales de cada grafo.

In [4]:
degrees = [list(nx.algorithms.degree_centrality(g).values()) for g in graphs]
betweenness = [list(nx.algorithms.betweenness_centrality(g).values()) for g in graphs]
closeness = [list(nx.algorithms.closeness_centrality(g).values()) for g in graphs]
eigenvector = [list(nx.algorithms.eigenvector_centrality(g, 500).values()) for g in graphs]
page_ranks = [list(nx.pagerank(g).values()) for g in graphs]
clustering_coeff = [list(nx.algorithms.cluster.clustering(g).values()) for g in graphs]
average_neighbor = [list(nx.average_neighbor_degree(g).values()) for g in graphs]

A continuación se crea una lista de listas booleanas que representan todas las posibles combinaciones sin repetición que se pueden hacer con las medidas de centralidad calculadas. Esta lista se utilizara para probar cuál combinación de medidas de centralidad dan los mejores resultados de predicción.

In [5]:
centrality_measures = ['degrees', 'betweenness', 'closeness', 'eigenvector', 'pagerank', 'clustering', 'average_neighbor']
parameter_combinations = []
for combination in range(1, 2 ** len(centrality_measures)):
    params = [bool(combination & (1 << i)) for i in range(len(centrality_measures))]
    parameter_combinations.append(params)

También se crea un array que contenga todas las medidas de centralidad de cada grafo.

In [6]:
X_metrics = []

for i in range (0, len(universities)):
    X_metrics.append(np.array([degrees[i], betweenness[i], closeness[i], eigenvector[i], page_ranks[i], clustering_coeff[i], average_neighbor[i]]).T)

A continuación se construye un clasificador random forest y se aplica la validación cruzada a cada grafo creado. Solo se utiliza el contenido del vector de palabras como parámetro X del dataset, con el objetivo de comparar el rendimiento al introducir los parámetros relacionales. Se utilizan las mismas métricas de rendimiento que en la investigación "Link Based Classification".

In [7]:
scores = []
scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']

for i in range(0,len(universities)):

    clf = RandomForestClassifier(n_estimators= 1000, criterion= "gini", random_state= 10052)
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        score = cross_validate(clf, np.array(word_vectors[i]), classifications[i], scoring=scoring, cv=10)

    scores.append(score)

for i in range(0, len(universities)):
    print(f'{universities[i].capitalize()} university:\n  Avg. Accuracy = {np.mean(scores[i]["test_accuracy"]):.3f},\n  Avg. F1 = {np.mean(scores[i]["test_f1_weighted"]):.3f},\n  Avg. Precision = {np.mean(scores[i]["test_precision_weighted"]):.3f},\n  Avg. Recall = {np.mean(scores[i]["test_recall_weighted"]):.3f}')

Cornell university:
  Avg. Accuracy = 0.712,
  Avg. F1 = 0.652,
  Avg. Precision = 0.635,
  Avg. Recall = 0.712
Texas university:
  Avg. Accuracy = 0.776,
  Avg. F1 = 0.715,
  Avg. Precision = 0.721,
  Avg. Recall = 0.776
Washington university:
  Avg. Accuracy = 0.817,
  Avg. F1 = 0.762,
  Avg. Precision = 0.735,
  Avg. Recall = 0.817
Wisconsin university:
  Avg. Accuracy = 0.811,
  Avg. F1 = 0.763,
  Avg. Precision = 0.741,
  Avg. Recall = 0.811


Se construye un clasificador de random forest y se aplica la validación cruzada, pero ahora utilizando una de los parámetros relacionales calculados.

In [8]:
scores = []

for i in range(0,len(universities)):

    clf = RandomForestClassifier(n_estimators= 1000, criterion= "gini", random_state= 10052)

    x = np.hstack((np.array(word_vectors[i]), np.array(eigenvector[i]).reshape(-1, 1)))

    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        score = cross_validate(estimator= clf, X = x, y= classifications[i], scoring=scoring, cv=10)
        
    scores.append(score)

for i in range(0, len(universities)):
    print(f'{universities[i].capitalize()} university:\n  Avg. Accuracy = {np.mean(scores[i]["test_accuracy"]):.3f},\n  Avg. F1 = {np.mean(scores[i]["test_f1_weighted"]):.3f},\n  Avg. Precision = {np.mean(scores[i]["test_precision_weighted"]):.3f},\n  Avg. Recall = {np.mean(scores[i]["test_recall_weighted"]):.3f}')


Cornell university:
  Avg. Accuracy = 0.727,
  Avg. F1 = 0.669,
  Avg. Precision = 0.651,
  Avg. Recall = 0.727
Texas university:
  Avg. Accuracy = 0.797,
  Avg. F1 = 0.743,
  Avg. Precision = 0.752,
  Avg. Recall = 0.797
Washington university:
  Avg. Accuracy = 0.817,
  Avg. F1 = 0.762,
  Avg. Precision = 0.735,
  Avg. Recall = 0.817
Wisconsin university:
  Avg. Accuracy = 0.815,
  Avg. F1 = 0.770,
  Avg. Precision = 0.757,
  Avg. Recall = 0.815


En primera instancia, no se aprecia una mejora en el rendimiento, de hecho, utilizando solo el vector de palabras se obtiene un mejor rendimiento en la mayoría de los scores que añadiendo el parámetro de eigenvector. A continuación se hará finetuning con RandomGridSearch al modelo con solo el vector de palabras, para encontrar los mejores parámetros, también se probarán las distintas combinaciones de métricas relacionales posibles y se encontrarán los parámetros que consigan los mejores resultados, también se hará finetuning a la combinación de métricas encontrada.

Primero se hará un randomized search en un rango grande de hiperparámetros, luego se explorará más a profundidad rangos cercanos a los encontrados por el randomized search inicial.

In [9]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = [ 'sqrt', 'auto', None, 0.3]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [int(x) for x in np.linspace(2, 410, num = 11)]
min_samples_leaf = [int(x) for x in np.linspace(1, 110, num = 11)]
bootstrap = [True, False]
criterion = ['gini', 'entropy', "log_loss"]
max_leaf_nodes = [int(x) for x in np.linspace(10, 110, num = 11)]
max_leaf_nodes.append(None)
min_impurity_decrease = [0.0, 0.0001, 0.001, 0.01, 0.1]
oob_score = [True, False]


random_wide_grid = {'n_estimators': n_estimators,
                    'max_features': max_features,
                    'max_depth': max_depth,
                    'min_samples_split': min_samples_split,
                    'min_samples_leaf': min_samples_leaf,
                    'bootstrap': bootstrap,
                    'criterion': criterion,
                    'max_leaf_nodes': max_leaf_nodes,
                    'min_impurity_decrease': min_impurity_decrease,
                    'oob_score': oob_score}
random_wide_grid

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'max_features': ['sqrt', 'auto', None, 0.3],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_samples_split': [2, 42, 83, 124, 165, 206, 246, 287, 328, 369, 410],
 'min_samples_leaf': [1, 11, 22, 33, 44, 55, 66, 77, 88, 99, 110],
 'bootstrap': [True, False],
 'criterion': ['gini', 'entropy', 'log_loss'],
 'max_leaf_nodes': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'min_impurity_decrease': [0.0, 0.0001, 0.001, 0.01, 0.1],
 'oob_score': [True, False]}

In [10]:
for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_wide_grid, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(np.array(word_vectors[i]), classifications[i])    
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Cornell university:
0.7005640164397542
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Texas university:
0.8036323662776752
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 40,
 'max_features': None,
 'max_leaf_nodes': 20,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 11,
 'min_samples_split': 42,
 'n_estimators': 1400,
 'oob_score': False}
 Washington universi

Se puede apreciar que los scores F1 mejoraron considerablemente en los 4 grafos. Ahora se procede a realizar un random grid search explorando más a profundidad la cercania de los valores obtenidos.

In [11]:
n_estimators = [400, 600, 800, 1000]
max_features = ['sqrt', 0.3, None]
max_depth = [20, 40, 60, 90, 100, 120]
min_samples_split = [1, 2, 5, 30, 42, 50]
min_samples_leaf = [1, 2, 5, 11, 15, 18]
bootstrap = [True, False]
criterion = ['entropy', 'gini']
max_leaf_nodes = [10, 20, 50, 80, 100, 120]
min_impurity_decrease = [0.0001, 0.001, 0.0]
oob_score = [True, False]

random_narrow_grid = {  'n_estimators': n_estimators,
                        'max_features': max_features,
                        'max_depth': max_depth,
                        'min_samples_split': min_samples_split,
                        'min_samples_leaf': min_samples_leaf,
                        'bootstrap': bootstrap,
                        'criterion': criterion,
                        'max_leaf_nodes': max_leaf_nodes,
                        'min_impurity_decrease': min_impurity_decrease,
                         'oob_score': oob_score}
random_narrow_grid

{'n_estimators': [400, 600, 800, 1000],
 'max_features': ['sqrt', 0.3, None],
 'max_depth': [20, 40, 60, 90, 100, 120],
 'min_samples_split': [1, 2, 5, 30, 42, 50],
 'min_samples_leaf': [1, 2, 5, 11, 15, 18],
 'bootstrap': [True, False],
 'criterion': ['entropy', 'gini'],
 'max_leaf_nodes': [10, 20, 50, 80, 100, 120],
 'min_impurity_decrease': [0.0001, 0.001, 0.0],
 'oob_score': [True, False]}

In [12]:
word_vector_fine_tuned_params = []          # Variable para guardar los mejores parámetros encontrados utilizando solo el vector de palabras

for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_narrow_grid, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(np.array(word_vectors[i]), classifications[i])    
    
    word_vector_fine_tuned_params.append(random_search.best_params_)
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': None,
 'max_leaf_nodes': 120,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1000,
 'oob_score': True}
 Cornell university:
0.7449254321455155
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': None,
 'max_leaf_nodes': 120,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1000,
 'oob_score': True}
 Texas university:
0.7667203235361131
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 120,
 'max_features': None,
 'max_leaf_nodes': 50,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 1,
 'min_samples_split': 42,
 'n_estimators': 600,
 'oob_score': False}
 Washington university:
0

Ahora se procede a encontrar la combinación de métricas relacionales que de el mejor resultado de rendimiento para luego hacerle finetuning a los parámetros de la combinación encontrada, asi como se hizo para el vector de palabras. Recordando que la variable X metrics es una lista de arrays cuyas columnas son las medidas de centralidad en el orden 'degrees', 'betweenness', 'closeness', 'eigenvector', 'pagerank', 'clustering', 'average_neighbor' de cada grafo.

En primera instancia se intentó realizar un grid search a cada combinación de métricas relacionales, pero el tiempo y la carga computacional de esto resultaba demasiado elevada. Por esto se decidió comentar el código y se probaran random forests con los parámetros default y una validación cruzada de 10 folds para todas las combinaciones, luego se hará un finetuning a la mejor combinación encontrada.

También se pudo apreciar que las métricas relacionales por si solas dan resultados mucho menor que 50%, por lo que solo se imprimiran y se guardarán las combinaciones con resultados mayores a cierto threshold. Más adelante, luego de realizar finetuning, se combinaran las métricas relacionales con el vector de palabras para ver su resultado conjunto.

In [13]:
best_measures = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
best_measures_indexes = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
best_scores = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
i = 0

for university in universities:

    for combination in parameter_combinations:

        measures = [centrality_measures[j] for j, include_measure in enumerate(combination) if include_measure]
        indexes = [j for j, include_measure in enumerate(combination) if include_measure]

        X_to_try = X_metrics[i][: , indexes]
        
        clf = RandomForestClassifier(random_state= 10052)
        #random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_wide_grid, n_iter= 100, cv= 5, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            score = cross_validate(clf, X_to_try, classifications[i], scoring=scoring, cv=10)
            #random_search.fit(X_to_try, classifications[i])  

        if np.mean(score["test_f1_weighted"]) > 0.45:
            #print(f'\n{university.capitalize()} university: Metricas: {measures}, Avg_F1_Score: {np.mean(score["test_f1_weighted"])}')
            best_measures[university].append(measures)
            best_measures_indexes[university].append(indexes)
            best_scores[university].append(np.mean(score["test_f1_weighted"]))
            
    combined = list(zip(best_scores[university], best_measures[university], best_measures_indexes[university]))
    sorted_combined = sorted(combined, key=lambda x: x[0], reverse=True)
    sorted_scores, sorted_parameters, sorted_indexes = zip(*sorted_combined)

    print(f'{university.capitalize()} university: \nMejores Scores: {sorted_scores[:5]} \nMejores parámetros: {sorted_parameters[:5]} \nIndices: {sorted_indexes[:5]}\n')
    i = 1 + i  

Cornell university: 
Mejores Scores: (0.496613488850331, 0.496208001945007, 0.4937696522603644, 0.4885837450714286, 0.4875150944957854) 
Mejores parámetros: (['betweenness', 'pagerank', 'clustering', 'average_neighbor'], ['degrees', 'betweenness', 'pagerank', 'average_neighbor'], ['betweenness', 'eigenvector', 'pagerank', 'clustering', 'average_neighbor'], ['degrees', 'betweenness', 'pagerank', 'clustering'], ['degrees', 'eigenvector', 'pagerank', 'average_neighbor']) 
Indices: ([1, 4, 5, 6], [0, 1, 4, 6], [1, 3, 4, 5, 6], [0, 1, 4, 5], [0, 3, 4, 6])

Texas university: 
Mejores Scores: (0.6404649524557992, 0.6333659907344118, 0.6286020918934726, 0.6260747957582435, 0.6199120385504826) 
Mejores parámetros: (['degrees', 'betweenness', 'closeness', 'eigenvector', 'clustering', 'average_neighbor'], ['degrees', 'betweenness', 'closeness', 'clustering', 'average_neighbor'], ['betweenness', 'closeness', 'eigenvector', 'clustering', 'average_neighbor'], ['degrees', 'betweenness', 'pagerank', '

Ahora se procede a hacer el mismo experimento pero introduciendo el vector de palabras junto con las combinaciones de las métricas relacionales.

In [14]:
best_measures = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
best_measures_indexes = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
best_scores = {"cornell": [], "texas": [], "washington":[], "wisconsin":[]}
i = 0

for university in universities:

    for combination in parameter_combinations:

        measures = [centrality_measures[j] for j, include_measure in enumerate(combination) if include_measure]
        indexes = [j for j, include_measure in enumerate(combination) if include_measure]

        X_to_try = X_metrics[i][: , indexes]
        X_combined = np.hstack((np.array(word_vectors[i]), X_to_try))
        clf = RandomForestClassifier(random_state= 10052)
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            score = cross_validate(clf, X_combined, classifications[i], scoring=scoring, cv=10)

        if np.mean(score["test_f1_weighted"]) > 0.45:
            best_measures_indexes[university].append(indexes)
            best_measures[university].append(measures)
            best_scores[university].append(np.mean(score["test_f1_weighted"]))
            
    combined = list(zip(best_scores[university], best_measures[university], best_measures_indexes[university]))
    sorted_combined = sorted(combined, key=lambda x: x[0], reverse=True)
    sorted_scores, sorted_parameters, sorted_indexes = zip(*sorted_combined)

    print(f'{university.capitalize()} university: \nMejores Scores: {sorted_scores[:5]} \nMejores parámetros: {sorted_parameters[:5]} \nIndices: {sorted_indexes[:5]}\n')
    i = 1 + i  

Cornell university: 
Mejores Scores: (0.6992677916881518, 0.6937679176010201, 0.6904978431012696, 0.6889780322017165, 0.6884253302596955) 
Mejores parámetros: (['closeness', 'pagerank', 'clustering'], ['degrees', 'betweenness', 'closeness', 'pagerank'], ['degrees', 'betweenness', 'closeness', 'pagerank', 'clustering', 'average_neighbor'], ['closeness', 'pagerank', 'average_neighbor'], ['betweenness', 'closeness', 'pagerank', 'clustering', 'average_neighbor']) 
Indices: ([2, 4, 5], [0, 1, 2, 4], [0, 1, 2, 4, 5, 6], [2, 4, 6], [1, 2, 4, 5, 6])

Texas university: 
Mejores Scores: (0.7607448249690812, 0.7591068697384487, 0.7555116637748216, 0.7536511149669045, 0.75283476951898) 
Mejores parámetros: (['degrees', 'eigenvector', 'pagerank', 'clustering', 'average_neighbor'], ['degrees', 'clustering', 'average_neighbor'], ['degrees', 'closeness'], ['degrees', 'betweenness', 'average_neighbor'], ['degrees', 'betweenness', 'closeness', 'eigenvector', 'pagerank']) 
Indices: ([0, 3, 4, 5, 6], [0, 

Ya obtenidas las mejores combinaciones de métricas relacionales se procede a hacer el mismo proceso de finetuning. Se hará un proceso para datasets creados únicamente con las métricas relacionales y otro proceso para datasets creados con las métricas relacionales más el vector de palabras. Primero se crean los datasets utilizando los indices de las mejores combinaciones de métricas encontradas para cada universidad.

In [15]:
best_metric_only_indexes = [[1, 4, 5, 6], [0, 1, 2, 3, 5, 6], [0, 2, 4], [1, 2, 6]]
best_metric_and_vector_indexes = [[2, 4, 5], [0, 3, 4, 5, 6], [0, 1, 2, 3, 5, 6], [0, 1, 2, 3, 4, 6]]
X_metric_only = []
X_metric_and_vector = []
i = 0

for indexes in best_metric_only_indexes:
    X_metric_only.append(X_metrics[i][: , indexes])
    i = i + 1

i = 0

for indexes in best_metric_and_vector_indexes:
    metrics_columns = X_metrics[i][: , indexes]
    X_combined = np.hstack((np.array(word_vectors[i]), metrics_columns))
    X_metric_and_vector.append(X_combined)
    i = i + 1


Se vuelve a utilizar el grid amplio de busqueda de parámetros para luego hacer una búsqueda más focalizada.

In [16]:
for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_wide_grid, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(X_metric_only[i] , classifications[i])    
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Cornell university:
0.49907545799573666
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Texas university:
0.6228393695041293
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Washington univer

Nuevamente se encontraron hiperparámetros parecidos para los 4 grafos. Se aplicará la una grid distinta a la del finetuning anterior ya que esta vez los 4 grafos arrojaron exactamente los mismos hiperparámetros.

In [17]:
n_estimators = [400, 600, 800, 1000]
max_features = ['sqrt', 0.3, None]
max_depth = [70, 80, 90, 100, 110]
min_samples_split = [1, 2, 5, 8]
min_samples_leaf = [1, 2, 5]
bootstrap = [True, False]
criterion = ['entropy', 'gini']
max_leaf_nodes = [80, 100, 120]
min_impurity_decrease = [0.0001, 0.001, 0.0]
oob_score = [True, False]

random__narrow_grid_2 = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'criterion': criterion,
               'max_leaf_nodes': max_leaf_nodes,
               'min_impurity_decrease': min_impurity_decrease,
               'oob_score': oob_score}

random__narrow_grid_2

{'n_estimators': [400, 600, 800, 1000],
 'max_features': ['sqrt', 0.3, None],
 'max_depth': [70, 80, 90, 100, 110],
 'min_samples_split': [1, 2, 5, 8],
 'min_samples_leaf': [1, 2, 5],
 'bootstrap': [True, False],
 'criterion': ['entropy', 'gini'],
 'max_leaf_nodes': [80, 100, 120],
 'min_impurity_decrease': [0.0001, 0.001, 0.0],
 'oob_score': [True, False]}

In [18]:
metrics_only_tuned_params = []          # Variable para guardar los mejores parámetros encontrados utilizando solo las métricas relacionales

for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random__narrow_grid_2, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(X_metric_only[i] , classifications[i])    
    
    metrics_only_tuned_params.append(random_search.best_params_)
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': None,
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 400,
 'oob_score': False}
 Cornell university:
0.5124263232738872
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': None,
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 8,
 'n_estimators': 400,
 'oob_score': False}
 Texas university:
0.6298609226183597
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': None,
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 400,
 'oob_score': False}
 Washington university:
0.55661808

Con esto se han obtenido los mejores hiperparámetros utilizando solamente las métricas relacionales. Ahora se procede a realizar el finetuning para el dataset que incluye las métricas relacionales y el vector de palabras a la vez. Nuevamente se utiliza el grid amplio de hiperparámetros.

In [19]:
for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_wide_grid, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(X_metric_and_vector[i] , classifications[i])    
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Cornell university:
0.6919972479044502
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 90,
 'max_features': 'sqrt',
 'max_leaf_nodes': 100,
 'min_impurity_decrease': 0.0001,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 600,
 'oob_score': False}
 Texas university:
0.7785916364337416
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 40,
 'max_features': None,
 'max_leaf_nodes': 20,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 11,
 'min_samples_split': 42,
 'n_estimators': 1400,
 'oob_score': False}
 Washington universi

Nuevamente se encontraron parametros parecidos para los 4 grafos. Esto puede deberse a que este conjunto de parámetros se adapta bien a la estructura de citas y vectores de palabra que tienen los grafos con lo que se está trabajando. Esta vez se encontraron exactamente los mismos parámetros que utilizando solo el vector de palabras, por lo que se usará el grid narrow 1.

In [20]:
vector_and_metrics_tuned_params = []          # Variable para guardar los mejores parámetros encontrados utilizando el vector de palabras y las métricas relacionales a la vez
for i in range(0,len(universities)):

    clf = RandomForestClassifier()
    random_search = RandomizedSearchCV(estimator=clf, param_distributions= random_narrow_grid, n_iter= 100, cv= 10, verbose= 1, random_state= 10052, scoring=scoring, refit="f1_weighted", n_jobs=-1)
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")
        random_search.fit(np.array(X_metric_and_vector[i]), classifications[i])    
    
    vector_and_metrics_tuned_params.append(random_search.best_params_)
    pprint.pprint(random_search.best_params_)
    print(f' {universities[i].capitalize()} university:')
    pprint.pprint(random_search.best_score_)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 120,
 'max_features': None,
 'max_leaf_nodes': 20,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 800,
 'oob_score': False}
 Cornell university:
0.7447484572531012
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': False,
 'criterion': 'gini',
 'max_depth': 120,
 'max_features': 0.3,
 'max_leaf_nodes': 10,
 'min_impurity_decrease': 0.001,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1000,
 'oob_score': False}
 Texas university:
0.7618242449558239
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
{'bootstrap': True,
 'criterion': 'entropy',
 'max_depth': 100,
 'max_features': None,
 'max_leaf_nodes': 120,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1000,
 'oob_score': True}
 Washington university:
0.80

Finalmente, se entrenará un Random Forest con los mejores hiperparámetros de cada dataset (vector de palabras, métricas relacionales y combinación de ambos) para una última comparación en base a la cuál realizar conclusiones en la memoria.

In [21]:
models = [word_vector_fine_tuned_params, metrics_only_tuned_params ,vector_and_metrics_tuned_params]
model_names = ['Solo vector de palabras', 'Solo métricas relacionales', 'Vector y métricas juntos:']
datasets = [word_vectors, X_metric_only, X_metric_and_vector]

In [22]:
for i in range(0,len(universities)):
    
    print(f'{universities[i].capitalize()} University:\n')
    
    for j in range(0,len(models)):

        clf = RandomForestClassifier(**models[j][i], random_state= 10052)
    
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            score = cross_validate(clf, np.array(datasets[j][i]), classifications[i], scoring=scoring, cv=10)
        
        print(f'    {model_names[j]}:')
        print(f'    Avg. Accuracy = {np.mean(score["test_accuracy"]):.3f}, Avg. F1 = {np.mean(score["test_f1_weighted"]):.3f}, Avg. Precision = {np.mean(score["test_precision_weighted"]):.3f},  Avg. Recall = {np.mean(score["test_recall_weighted"]):.3f}\n')

Cornell University:

    Solo vector de palabras:
    Avg. Accuracy = 0.754, Avg. F1 = 0.730, Avg. Precision = 0.747,  Avg. Recall = 0.754

    Solo métricas relacionales:
    Avg. Accuracy = 0.565, Avg. F1 = 0.512, Avg. Precision = 0.500,  Avg. Recall = 0.565

    Vector y métricas juntos::
    Avg. Accuracy = 0.763, Avg. F1 = 0.739, Avg. Precision = 0.750,  Avg. Recall = 0.763

Texas University:

    Solo vector de palabras:
    Avg. Accuracy = 0.787, Avg. F1 = 0.754, Avg. Precision = 0.764,  Avg. Recall = 0.787

    Solo métricas relacionales:
    Avg. Accuracy = 0.670, Avg. F1 = 0.634, Avg. Precision = 0.637,  Avg. Recall = 0.670

    Vector y métricas juntos::
    Avg. Accuracy = 0.803, Avg. F1 = 0.766, Avg. Precision = 0.769,  Avg. Recall = 0.803

Washington University:

    Solo vector de palabras:
    Avg. Accuracy = 0.796, Avg. F1 = 0.797, Avg. Precision = 0.833,  Avg. Recall = 0.796

    Solo métricas relacionales:
    Avg. Accuracy = 0.574, Avg. F1 = 0.532, Avg. Precision = 